In [1]:
import os

# Set SPARK_HOME and JAVA_HOME environment variables
os.environ['SPARK_HOME'] = '/usr/local/Cellar/apache-spark/3.5.1/libexec'
os.environ['JAVA_HOME'] = '/usr/local/opt/openjdk/libexec/openjdk.jdk/Contents/Home'

In [1]:
# Partial Bucket Sort

from pyspark.sql.functions import col, udf, row_number
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
from pyspark.sql.window import Window

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Sample Spark Program") \
    .getOrCreate()

# Data with two columns
data = [(29, 'A'), (25, 'B'), (3, 'C'), (49, 'A'), (9, 'B'), (37, 'C'), (21, 'A'), (43, 'B')]
df = spark.createDataFrame(data, ["value", "category"])

# Number of buckets - this can be parameterized or set based on your requirement
num_buckets = 5

# Calculate the maximum value from the DataFrame
max_value = df.agg({"value": "max"}).collect()[0][0]

# Example function to assign buckets
def assign_bucket(value, num_buckets, max_value):
    size = max_value / num_buckets # Calculate the size of each bucket based on the number of buckets and the maximum value in the data 
    return int(value / size) # Here we are using integer division to get the bucket number for the value 

# Register UDF with the computed max_value
bucket_udf = udf(lambda x: assign_bucket(x, num_buckets, max_value), IntegerType())

# Adding a bucket column
df = df.withColumn("bucket", bucket_udf(col("value")))

# Repartition based on the bucket column
df = df.repartition("bucket")

# Sort within each partition
# df = df.sortWithinPartitions("value")

# Define window specification
window_spec = Window.partitionBy("bucket").orderBy("value")

# Apply a window function like row_number to add a sequential row number within each partition
df = df.withColumn("row_number", row_number().over(window_spec))

df.show()
# # Collecting results to show
# sorted_df = df.drop("bucket").collect()

# # Print the results
# for row in sorted_df:
#     print(row)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/21 11:32:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----+--------+------+----------+
|value|category|bucket|row_number|
+-----+--------+------+----------+
|    3|       C|     0|         1|
|    9|       B|     0|         2|
|   21|       A|     2|         1|
|   25|       B|     2|         2|
|   29|       A|     2|         3|
|   37|       C|     3|         1|
|   43|       B|     4|         1|
|   49|       A|     5|         1|
+-----+--------+------+----------+



24/08/21 11:32:56 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
def partial_bucket_sort_strings(data, num_buckets, sort_threshold):
    """
    Perform partial bucket sort on a list of strings.
    
    Parameters:
    - data: List of strings to sort
    - num_buckets: Number of buckets to use
    - sort_threshold: Number of items in a bucket before it gets fully sorted
    
    Returns:
    - Sorted list of strings
    """
    
    # Step 1: Create buckets based on string lengths
    min_length = min(len(s) for s in data)
    max_length = max(len(s) for s in data)
    bucket_range = (max_length - min_length + 1) / num_buckets
    
    # Initialize buckets
    buckets = [[] for _ in range(num_buckets)]
    
    # Distribute data into buckets
    for s in data:
        length = len(s)
        index = int((length - min_length) / bucket_range)
        if index == num_buckets:  # Handle the edge case
            index -= 1
        buckets[index].append(s)
    
    # Step 2: Sort buckets partially
    sorted_data = []
    for bucket in buckets:
        if len(bucket) > sort_threshold:
            # Fully sort this bucket
            sorted_data.extend(sorted(bucket))
        else:
            # Sort partially (for simplicity, sorting the whole bucket here)
            sorted_data.extend(sorted(bucket))
    
    # Step 3: Merge buckets
    return sorted_data

# Example usage
data = ["apple", "banana", "kiwi", "cherry", "blueberry", "date"]
num_buckets = 3
sort_threshold = 2

sorted_data = partial_bucket_sort_strings(data, num_buckets, sort_threshold)
print("Sorted data (strings):", sorted_data)

Sorted data (strings): ['apple', 'date', 'kiwi', 'banana', 'cherry', 'blueberry']


In [3]:
from datetime import datetime

def partial_bucket_sort_dates(data, num_buckets, sort_threshold):
    """
    Perform partial bucket sort on a list of dates.
    
    Parameters:
    - data: List of datetime objects to sort
    - num_buckets: Number of buckets to use
    - sort_threshold: Number of items in a bucket before it gets fully sorted
    
    Returns:
    - Sorted list of dates
    """
    
    # Step 1: Create buckets based on year
    min_year = min(d.year for d in data)
    max_year = max(d.year for d in data)
    bucket_range = (max_year - min_year + 1) / num_buckets
    
    # Initialize buckets
    buckets = [[] for _ in range(num_buckets)]
    
    # Distribute data into buckets
    for d in data:
        year = d.year
        index = int((year - min_year) / bucket_range)
        if index == num_buckets:  # Handle the edge case
            index -= 1
        buckets[index].append(d)
    
    # Step 2: Sort buckets partially
    sorted_data = []
    for bucket in buckets:
        if len(bucket) > sort_threshold:
            # Fully sort this bucket
            sorted_data.extend(sorted(bucket))
        else:
            # Sort partially (for simplicity, sorting the whole bucket here)
            sorted_data.extend(sorted(bucket))
    
    # Step 3: Merge buckets
    return sorted_data

# Example usage
data = [
    datetime(2024, 8, 1),
    datetime(2023, 5, 12),
    datetime(2024, 1, 23),
    datetime(2023, 11, 30),
    datetime(2022, 12, 15)
]
num_buckets = 2
sort_threshold = 2

sorted_data = partial_bucket_sort_dates(data, num_buckets, sort_threshold)
print("Sorted data (dates):", sorted_data)

Sorted data (dates): [datetime.datetime(2022, 12, 15, 0, 0), datetime.datetime(2023, 5, 12, 0, 0), datetime.datetime(2023, 11, 30, 0, 0), datetime.datetime(2024, 1, 23, 0, 0), datetime.datetime(2024, 8, 1, 0, 0)]


In [4]:
from collections import defaultdict

def partial_bucket_sort_categorical(data, categories, sort_threshold):
    """
    Perform partial bucket sort on categorical data.
    
    Parameters:
    - data: List of categorical data (strings)
    - categories: List of categories to create buckets
    - sort_threshold: Number of items in a bucket before it gets fully sorted
    
    Returns:
    - Sorted list of categorical data
    """
    
    # Create buckets based on categories
    buckets = defaultdict(list)
    for item in data:
        if item in categories:
            buckets[item].append(item)
    
    # Step 2: Sort buckets partially
    sorted_data = []
    for category in categories:
        bucket = buckets[category]
        if len(bucket) > sort_threshold:
            # Fully sort this bucket
            sorted_data.extend(sorted(bucket))
        else:
            # Sort partially (for simplicity, sorting the whole bucket here)
            sorted_data.extend(sorted(bucket))
    
    # Step 3: Merge buckets
    return sorted_data

# Example usage
data = ["apple", "banana", "cherry", "apple", "banana", "date", "cherry"]
categories = ["apple", "banana", "cherry", "date"]
sort_threshold = 1

sorted_data = partial_bucket_sort_categorical(data, categories, sort_threshold)
print("Sorted data (categorical):", sorted_data)

Sorted data (categorical): ['apple', 'apple', 'banana', 'banana', 'cherry', 'cherry', 'date']
